In [2]:
import numpy as np
import tensorflow as tf
import warnings 

warnings.filterwarnings('ignore')

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

<em> Instead of manually running the initializer for every single variable, you can use the
global_variables_initializer() function. Note that it does not actually perform
the initialization immediately, but rather creates a node in the graph that will initialize
all variables when it is run:

In [6]:
init = tf.global_variables_initializer() # prepare an init node 

with tf.Session() as sass:
    init.run() # actually initilize all variables 
    result = f.eval()

In [7]:
## Interactive session 

sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [8]:
# Any node you create is automatically added to the default graph:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
# Creating a new graph and running it temporarily as the default graph 
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [10]:
## Lifecycle of a node 

w = tf.constant(3)
x = w + 2 
y = x + 5 
z = x * 3 

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


<em> If you want to evaluate y and z efficiently, without evaluating w and x twice as in the
previous code, you must ask TensorFlow to evaluate both y and z in just one graph
run

In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


### Linear regression in Tf

In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [13]:
housing = fetch_california_housing()

In [14]:
m, n = housing.data.shape # m = no_samples,  n = no_features
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)

In [15]:
# By using the normal equation 

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) 

In [16]:
with tf.Session() as sess:
    theta.eval()

### Implementing Gradient Descent 

Manually Computing the gradient 

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [33]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [19]:
init = tf.global_variables_initializer()

In [20]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0 :
            print('Epoch ', epoch, "MSE : ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch  0 MSE :  5.6784286
Epoch  100 MSE :  4.8873243
Epoch  200 MSE :  4.8512945
Epoch  300 MSE :  4.8374257
Epoch  400 MSE :  4.8280025
Epoch  500 MSE :  4.821221
Epoch  600 MSE :  4.816312
Epoch  700 MSE :  4.812755
Epoch  800 MSE :  4.810176
Epoch  900 MSE :  4.8083034


### Using Autodiff

In [21]:
gradients = tf.gradients(mse, [theta])[0]

In [22]:
training_op = tf.assign(theta, theta - learning_rate*gradients)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0 :
            print('Epoch ', epoch, "MSE : ", mse.eval())
        sess.run(training_op)

Epoch  0 MSE :  5.6784286
Epoch  100 MSE :  4.8873243
Epoch  200 MSE :  4.8512945
Epoch  300 MSE :  4.8374257
Epoch  400 MSE :  4.8280025
Epoch  500 MSE :  4.821221
Epoch  600 MSE :  4.816312
Epoch  700 MSE :  4.8127556
Epoch  800 MSE :  4.810175
Epoch  900 MSE :  4.8083034


#### Using Gradient descent optimizer 

In [39]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            summary_str = mse_summary.eval()
            step = epoch
            file_writer.add_summary(summary_str, step)
        if epoch % 100 == 0:
            print('Epoch ', epoch, "MSE : ", mse.eval())
        sess.run(training_op)
        
file_writer.close()

Epoch  0 MSE :  5.6784286
Epoch  100 MSE :  4.8873243
Epoch  200 MSE :  4.8512945
Epoch  300 MSE :  4.8374257
Epoch  400 MSE :  4.8280025
Epoch  500 MSE :  4.821221
Epoch  600 MSE :  4.816312
Epoch  700 MSE :  4.8127556
Epoch  800 MSE :  4.810175
Epoch  900 MSE :  4.8083034


### Placeholders

In [23]:
## Placeholders, typically used to pass the training data to TensorFlow during training

A = tf.placeholder(dtype=tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [24]:
print(B_val_1)

[[6. 7. 8.]]


In [25]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


#### Mini Batch Gradient Descent 


In [82]:
n_epochs = 1000
learning_rate = 0.01 

In [115]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')

In [112]:
theta = tf.Variable(tf.random_uniform(shape=[n+1, 1], minval=-1.0, maxval=1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [ ]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [52]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

In [111]:
best_theta

array([[ 0.9045429 ],
       [ 0.8377844 ],
       [ 0.10645497],
       [-0.25947902],
       [ 0.29196444],
       [ 0.00181689],
       [ 0.2128084 ],
       [-0.89034677],
       [-0.85242176]], dtype=float32)

### Saving a model 

<em>Just create a Saver node at
the end of the construction phase (after all variable nodes are created); then, in the
execution phase, just call its save() method whenever you want to save the model,
passing it the session and path of the checkpoint file:

In [26]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 5.6784286
Epoch 100 MSE = 4.8873243
Epoch 200 MSE = 4.8512945
Epoch 300 MSE = 4.8374257
Epoch 400 MSE = 4.8280025
Epoch 500 MSE = 4.821221
Epoch 600 MSE = 4.816312
Epoch 700 MSE = 4.8127556
Epoch 800 MSE = 4.810175
Epoch 900 MSE = 4.8083034


In [27]:
best_theta

array([[ 9.0454292e-01],
       [ 7.7407807e-01],
       [ 1.3119239e-01],
       [-1.1784511e-01],
       [ 1.6477819e-01],
       [ 7.4408075e-04],
       [-3.9194513e-02],
       [-8.6135650e-01],
       [-8.2347965e-01]], dtype=float32)

To restore the model

In [28]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() 

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [29]:
best_theta_restored

array([[ 9.0454292e-01],
       [ 7.7407807e-01],
       [ 1.3119239e-01],
       [-1.1784511e-01],
       [ 1.6477819e-01],
       [ 7.4408075e-04],
       [-3.9194513e-02],
       [-8.6135650e-01],
       [-8.2347965e-01]], dtype=float32)

### Tensorboard Visualizations

<em>You need to use a different log directory every time you
run your program, or else TensorBoard will merge stats from different runs

In [8]:
tf.reset_default_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [45]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [46]:
# Namespaces are used to avoid the graph cluttering of the graphs. 

with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')

In [47]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [48]:
n_epochs = 10
gbatch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [49]:
# Next, add the following code at the very end of the construction phase:
"""creates a node in the graph that will evaluate the MSE value and write it
to a TensorBoard-compatible binary log string called a summary"""
mse_summary = tf.summary.scalar('MSE', mse)

"""creates a FileWriter that you will use to write summaries to logfiles in the log directory.
The first parameter indicates the path of the log directory (in this case something like
tf_logs/run-20160906091959/, relative to the current directory). The second
(optional) parameter is the graph you want to visualize"""
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

<br><em>Update the execution phase to evaluate the mse_summary node regularly
during training (e.g., every 10 mini-batches). This will output a summary that
you can then write to the events file using the file_writer.

In [53]:
with tf.Session() as sess:                                                        
    sess.run(init)                                                                

    for epoch in range(n_epochs):                                                 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     

In [54]:
file_writer.close()

## Modularity 

<em>Suppose you want to create a graph that adds the output of two rectified linear units
(ReLU). A ReLU computes a linear function of the inputs, and outputs the result if it
is positive, and 0 otherwise,

In [67]:
n_features = 3 
X = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='X')

w1 = tf.Variable(tf.random_normal(shape=(n_features, 1)), name='w1')
w2 = tf.Variable(tf.random_normal(shape=(n_features, 1)), name='w2')

b1 = tf.Variable(0.0, name='b1')
b2 = tf.Variable(0.0, name='b2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0.0, name='relu1')
relu2 = tf.maximum(z2, 0.0, name='relu2')

output = tf.add(relu1, relu2, name='output')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(output, feed_dict={X: np.array([2, 3, 5]).reshape(-1, 3)}))

[[12.849395]]


In [12]:
## Without copy pasting like this, you can do this like below 

def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name='relu')

In [13]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='X')
threshold = tf.Variable(0.0, name='threshold')

relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name='output')

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(output, feed_dict={X:  np.array([2, 3, 5]).reshape(-1, 3)})
    file_writer = tf.summary.FileWriter(logdir, graph=sess.graph)
    
file_writer.close()